# Inverse Kinematics

## Analytic Inverse Kinematics

<center>
    $T(\theta)=e^{[S_1]\theta_1}e^{[S_2]\theta_2}e^{[S_3]\theta_3}e^{[S_4]\theta_4}e^{[S_5]\theta_5}e^{[S_6]\theta_6}$ (6R 6DOF)
</center>

Give some end-effector frame $X\in SE(3)$, the inverse kinematics problem is to find a solution $\theta \in R^6$ satisfying $T(\theta)=X$

## Numerical Inverse Kinematics

### Newton-Raphson Method

To solve the equation "$g(\theta)=0$" numerically for a given differentiable function g:R$\rightarrow$R, assume $\theta^0$ is an initial guess for the solution. Write the Taylor expansion of $g(\theta)$ at $\theta^0$ and truncate it at first order:

<center>
    $g(\theta)=g(\theta^0)+\frac{\partial{g(\theta)}}{\partial{\theta}}(\theta-\theta^0)+higher-order\;terms(h.o.t)$<br><br>
    $\theta=\theta^0-(\frac{\partial{g(\theta^0)}}{\partial{\theta}})^{-1}g(\theta^0)$<br><br>
    $\theta{k+1}=\theta^k-(\frac{\partial{g(\theta^k)}}{\partial{\theta}})^{-1}g(\theta^k)$
</center>

The iteration is repeated until some stopping criterion is satisfied $|g(\theta^k)-g(\theta^{k+1})|/|g(\theta^k)|\leq \epsilon$.

### Numerical Inverse Kinematics Algorithm

Suppose we express the end-effector frame using a coordinate vector x governed by the forward kinematics $x=f(\theta)$, a nonlinear vector equation mapping the "n"-joint coordinates to the "m" end-effector coordinates. Assume that $f:R^n\rightarrow R^m$ is differentiable, and let "$x_d$" be the desired end-effector coordinates. Then $g(\theta)$ for the Newton-Raphson method is defined as $g(\theta)=x_d-f(\theta)$, and the goal is to find joint coordinates $\theta_d$ such that<br><br>
<center>$g(\theta_d)=x_d-f(\theta_d)=0$</center>

<img src='img/Img18.png'>

Given an initial guess $\theta^0$ which is "close to" a solution $\theta_d$, the kinematics can be expressed as a Taylor expansion

<center>
    $x_d=f(\theta_d)=f(\theta^0)+\frac{\partial{f(\theta_d-\theta^0)}}{\partial{\theta}}+h.o.t$<br><br>
    $\frac{\partial{f(\theta_d-\theta^0)}}{\partial{\theta}}=J(\theta^0)$
</center>

where $J(\theta^0)\in R^{mxn}$ is the coordinate Jacobian evaluated at $\theta^0$. Truncating the Taylor expansion at first order:

<center>$J(\theta^0)\triangle\theta=x_d-f(\theta^0)$

Assuming that the Jacobian is square and invertible, we can solve for $\triangle\theta$ as

<center>$\triangle\theta=J^{-1}(\theta^0)(x_d-f(\theta^0))$

If J is not invertible, either because is not square or because it's singular. Equation "$J(\theta^0)\triangle\theta=x_d-f(\theta^0)$" can be solve for $\triangle\theta$ by replacing $J^{-1}$ with the Moore-Penrose pseudoinverse $J^{\dagger}$. For any equation of the form $J_y=z$, where $J\in R^{mxn},\; y\in R^n$ and $x\in R^m$, the solution

<center>
    $y^{*}=J^{\dagger}z$<br><br>
    $\triangle\theta=J^{\dagger}(\theta^0)(x_d-f(\theta^0))$
</center>

This suggests using the Newton-Raphson algorithm for finding $\theta_d$:<br><br>
(a) Initialization: Given $x_d\in R^m$ and an initial guess $\theta^0 \in R^n$, set i=0.<br>
(b) Set $e=x_d-f(\theta^i)$. While $||e||>\epsilon$ for some small $\epsilon$<br>
- Set $\theta^{i+1}=\theta^i+J^{\dagger}(\theta^i)e$
- Increment i.

To modify this algorithm to work with a desired end-effector configuration represented as $T_{sd}\in SE(3)$ instead of as a coordinate vector $x_d$, we can replace the coordinate J with the end-effector of $J_b\in R^{6xn}$. Note that the vector $e=x_d-f(\theta^i)$, representing the direction form the current guess to the desired end-effector configuration, cannot simply be replaced by $T_{sd}-T_{sb}(\theta^i)$. To find the right analogy, we should think of $e=x_d-f(\theta^i)$ as a velocity vector which would cause a motion from $f(\theta^i)$ to $x_d$. Similarly, we should look for a $V_b$ which would cause a motion from $T_{sb}(\theta^i)$ to the desired configuration $T_{sd}$

<center>
    $T_{bd}(\theta^i)=T_{sb}^{-1}(\theta^i)T_{sd}=T_{bs}(\theta^i)T_{sd}$<br><br>
    $[V_b]=log (T_{bd}(\theta^i))$
</center>

This leads to the following inverse kinematics algorithm:

(a) Initialization: Given $T_{sd}$ and a initial guess $\theta^0 \in R^n$, set i=0.<br><br>
(b) Set $[V_b]=log(T_{sb}^-1(\theta^i)T_{sd})$. While $||w_b||>\epsilon_w$ or $||v_b||>\epsilon_v$ for small $\epsilon_w,\epsilon_v$.
- Set $\theta^{i+1}=\theta^i+J^{\dagger}_b(\theta^i)V_b$
- Increment i.